# Data Preparation

In [1]:
# Imports and defines

from neurallib.clean import *

project = '540_nan_copy'
results_folder = f"./results/{project}/"
in_folder = f".../infiles/{project}/"

In [2]:
# Read in UV
df = pd.read_csv(f'infiles/uv.csv')

# Prepare ET data

In [3]:
# Read in and merge raw ET data
# keep = {
#     'Study Name':'subset_old',
#     'Respondent Name':'Respondent Name',
#     'Label':'AOI',
#     'Parent Label':'Stim',
#     'Dwell count':'FixationCount',
#     'First fixation duration':'FFD',
#     'TTFF (AOI)':'TTFF',
#     'Dwell time (fixation, ms)':'TFD',
#     'Mouse click count':'Click'
# }

# results = []
# files = get_files('infiles/ET/')
# for file in files:
#     data,_ = read_imotions(f'infiles/ET/{file}')
#     data = data.loc[data['Type']=='Static AOI']
#     data = data[keep.keys()]
#     data = data.rename(keep,axis = 1)
#     data = data.replace('NaN',np.nan)
#     results.append(data)

# df_et = pd.concat(results)
# df_et = df_et.dropna(subset=['Respondent Name'])


# for res, res_data in df_et.groupby('Respondent Name'):
#     try:
#         df_et.loc[df_et['Respondent Name']==res, 'resp_id_old'] = int(res.split('_')[1])
#     except:
#         df_et.loc[df_et['Respondent Name']==res, 'resp_id_old'] = int(res.split(' ')[1])

# resp_key = pd.read_csv('infiles/Keys/nandos_resp_id_new.csv')
# df_et = pd.merge(df_et,resp_key, on = ['resp_id_old','subset_old'])
df_et = pd.read_csv('infiles/df_et.csv')

    

In [4]:
# Set Accuracy flag for findability task based on Clicks

#Define list of AOIS to look for for each item
items_to_find = {
    1:{'Item1':73,
     'Item2':17,
     'Item3':61,
     'Item4':10,
     'Item5':30},
     2:{'Item1':9,
     'Item2':17,
     'Item3':61,
     'Item4':10,
     'Item5':30},
     3:{'Item1':9,
     'Item2':17,
     'Item3':61,
     'Item4':10,
     'Item5':30},
}


for index, row in df.iterrows():
    for i in range(1,6):
        try:
            clicked = df_et.loc[ (df_et.respondent == row.respondent)
                                &(df_et.Stim.str.contains(f'Item{i}'))
                                &(df_et.AOI==items_to_find[row.subset][f'Item{i}'])]['Click'].values[0]
            clicked = 1 if clicked>0 else 0
        except:
            clicked = np.nan
        df.loc[index, f'iam_findSpecific_Item{i}_ClickAccurate']=clicked

In [5]:
# Create df_item

df_item = df_et.copy()

df_item = df_item.loc[df_item['AOI'].str.contains('Item')]

for res, res_data in df_item.groupby('AOI'):
    try:
        df_item.loc[df_item['AOI']==res, 'AOI'] = int(res.split('_')[1])
    except:
        print(f'Failed {res}')

In [6]:
# Set Subj. Undiscovered in AOI data

item_key = pd.read_csv('infiles/Keys/items.csv')
df_fam = pd.read_csv(f'results/df_fam.csv')


result = []
for item in item_key.ID.unique():
    try:
        fam_id = item_key.loc[item_key.ID==item]['FAM'].values[0]
        for respondent in df_item.respondent.unique():
            try:
                df_item.loc[(df_item.respondent==respondent) & (df_item.AOI==int(item)), 'SubjUndiscovered'] = df_fam.loc[df_fam.respondent==respondent][f"VALUE_{fam_id}_Question (2)" ].values[0]
            except:
                print(f'DNF {fam_id} for respondent {respondent}')
    except:
        print(f'DNF {fam_id} for item {item}')

DNF FAM_01 for respondent 11
DNF FAM_01 for respondent 10
DNF FAM_01 for respondent 72
DNF FAM_01 for respondent 74
DNF FAM_01 for respondent 42
DNF FAM_01 for respondent 34
DNF FAM_02 for respondent 11
DNF FAM_02 for respondent 10
DNF FAM_02 for respondent 72
DNF FAM_02 for respondent 74
DNF FAM_02 for respondent 42
DNF FAM_02 for respondent 34
DNF FAM_03 for respondent 11
DNF FAM_03 for respondent 10
DNF FAM_03 for respondent 72
DNF FAM_03 for respondent 74
DNF FAM_03 for respondent 42
DNF FAM_03 for respondent 34
DNF FAM_04 for respondent 11
DNF FAM_04 for respondent 10
DNF FAM_04 for respondent 72
DNF FAM_04 for respondent 74
DNF FAM_04 for respondent 42
DNF FAM_04 for respondent 34
DNF FAM_05 for respondent 11
DNF FAM_05 for respondent 10
DNF FAM_05 for respondent 72
DNF FAM_05 for respondent 74
DNF FAM_05 for respondent 42
DNF FAM_05 for respondent 34
DNF FAM_06 for respondent 11
DNF FAM_06 for respondent 10
DNF FAM_06 for respondent 72
DNF FAM_06 for respondent 74
DNF FAM_06 for

In [7]:
# Calculate metrics for undiscovered items


find_anything_stims = {'Slide8': 'FrontPage',
                       'Slide13': 'BackPage'}

undiscovered_items = item_key.loc[item_key['Undiscovered']==1]['ID'].to_list()

for index, row in df.iterrows():
    for stim,label in find_anything_stims.items():

        try:
            TTFF = df_item.loc[ (df_item.respondent == row.respondent)
                                &(df_item.Stim.str.contains(stim))
                                &(df_item.AOI.isin(undiscovered_items))]['TTFF'].sort_values(ascending=True).values[0]
            TFD = df_item.loc[ (df_item.respondent == row.respondent)
                                &(df_item.Stim.str.contains(stim))
                                &(df_item.AOI.isin(undiscovered_items))]['TFD'].sum()
            count = df_item.loc[ (df_item.respondent == row.respondent)
                                &(df_item.Stim.str.contains(stim))
                                &(df_item.AOI.isin(undiscovered_items))]['TTFF'].count()
        except:
            TTFF = np.nan
            TFD = np.nan
            count = np.nan

        df.loc[index, f'iam_findAnything_{label}_Undiscovered_TTFF']=TTFF
        df.loc[index, f'iam_findAnything_{label}_Undiscovered_TFD']=TFD
        df.loc[index, f'iam_findAnything_{label}_Undiscovered_FixationCount']=count

for index, row in df.iterrows():
    for stim,label in find_anything_stims.items():
        try:
            TTFF = df_item.loc[ (df_item.respondent == row.respondent)
                                &(df_item.Stim.str.contains(stim))
                                &(df_item.AOI.isin(undiscovered_items))
                                &(df_item.SubjUndiscovered==1)]['TTFF'].sort_values(ascending=True).values[0]
            TFD = df_item.loc[ (df_item.respondent == row.respondent)
                                &(df_item.Stim.str.contains(stim))
                                &(df_item.AOI.isin(undiscovered_items))
                                &(df_item.SubjUndiscovered==1)]['TFD'].sum()
            count = df_item.loc[ (df_item.respondent == row.respondent)
                                &(df_item.Stim.str.contains(stim))
                                &(df_item.AOI.isin(undiscovered_items))
                                &(df_item.SubjUndiscovered==1)]['TTFF'].count()
        except:
            TTFF = np.nan
            TFD = np.nan
            count = np.nan
            
        df.loc[index, f'iam_findAnything_{label}_SubjUndiscovered_TTFF']=TTFF
        df.loc[index, f'iam_findAnything_{label}_SubjUndiscovered_TFD']=TFD
        df.loc[index, f'iam_findAnything_{label}_SubjUndiscovered_FixationCount']=count


In [8]:
# Calculate GP data

df_gp = df_item.copy()
df_gp = df_gp.loc[df_gp.Stim.str.contains('Stim')]

# Define top 5 high-gp items

high_gp = {'1/4 Chicken':[2,3,4,5],
           'Full Chicken':[36,79,80,81,82],
           'Bowls and Salads':[57,11,39,69,43,42,78,32],
           'Sides':[7,83,20,29,54,60,61,35,51,53,45],
           '1/2 Chicken':[0,1],
           'Sosaties':[15,16,74,17,98]
        }

for index, row in df.iterrows():
    for stim in df_gp.Stim.unique():
        for category,items in high_gp.items():
            try:
                TTFF = df_gp.loc[ (df_gp.respondent == row.respondent)
                                    &(df_gp.Stim.str.contains(stim))
                                    &(df_gp.AOI.isin(items))]['TTFF'].sort_values(ascending=True).values[0]
                TFD = df_gp.loc[ (df_gp.respondent == row.respondent)
                                    &(df_gp.Stim.str.contains(stim))
                                    &(df_gp.AOI.isin(items))]['TFD'].mean()
                count = df_gp.loc[ (df_gp.respondent == row.respondent)
                                    &(df_gp.Stim.str.contains(stim))
                                    &(df_gp.AOI.isin(items))]['TTFF'].count()
            except:
                TTFF = np.nan
                TFD = np.nan
                count = np.nan

            df.loc[index, f'iam_standard_{stim}_HighGP_{category}_TTFF']=TTFF
            df.loc[index, f'iam_standard_{stim}_HighGP_{category}_TFD']=TFD
#             df.loc[index, f'iam_standard_{stim}_HighGP_{category}_FixationCount']=count

In [9]:
# Calculate New GP data

df_gp = df_item.copy()
df_gp = df_gp.loc[df_gp.Stim.str.contains('Stim')]

# Define top 5 high-gp items

high_gp = {'Garlic/Nandos Roll':[12,37,114,],
           'Strips and Rice':[22],
           'All-in-one bowl':[57],
           'Avo and Halloumi bowl':[11],
           'Strips Veg and Rice':[78]
        }

for index, row in df.iterrows():
    for stim in df_gp.Stim.unique():
        for category,items in high_gp.items():
            try:
                TTFF = df_gp.loc[ (df_gp.respondent == row.respondent)
                                    &(df_gp.Stim.str.contains(stim))
                                    &(df_gp.AOI.isin(items))]['TTFF'].sort_values(ascending=True).values[0]
                TFD = df_gp.loc[ (df_gp.respondent == row.respondent)
                                    &(df_gp.Stim.str.contains(stim))
                                    &(df_gp.AOI.isin(items))]['TFD'].mean()
                count = df_gp.loc[ (df_gp.respondent == row.respondent)
                                    &(df_gp.Stim.str.contains(stim))
                                    &(df_gp.AOI.isin(items))]['TTFF'].count()
            except:
                TTFF = np.nan
                TFD = np.nan
                count = np.nan

            df.loc[index, f'iam_standard_{stim}_NewGP_{category}_TTFF']=TTFF
            df.loc[index, f'iam_standard_{stim}_NewGP_{category}_TFD']=TFD
#             df.loc[index, f'iam_standard_{stim}_HighGP_{category}_FixationCount']=count

C:\Users\ashra\AppData\Local\Temp\ipykernel_39252\2384716128.py:33: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df.loc[index, f'iam_standard_{stim}_NewGP_{category}_TTFF']=TTFF
C:\Users\ashra\AppData\Local\Temp\ipykernel_39252\2384716128.py:34: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df.loc[index, f'iam_standard_{stim}_NewGP_{category}_TFD']=TFD
C:\Users\ashra\AppData\Local\Temp\ipykernel_39252\2384716128.py:33: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many

In [11]:
# Calculate saliency metrics

df_sal = df_et.copy().loc[(~df_et.AOI.str.contains('Item'))
                            &(df_et.Stim.str.contains('Stim'))]

for index, row in df.iterrows():
    for stim, df_stim in df_sal.groupby('Stim'):
        for aoi in df_stim.AOI.unique():
            try:
                TTFF = df_stim.loc[ (df_stim.respondent == row.respondent)
                                    &(df_stim.AOI == aoi)]['TTFF'].sort_values(ascending=True).values[0]
                TFD = df_stim.loc[ (df_stim.respondent == row.respondent)
                                    &(df_stim.AOI == aoi)]['TFD'].sum()
                count = df_stim.loc[ (df_stim.respondent == row.respondent)
                                    &(df_stim.AOI == aoi)]['TTFF'].count()
            except:
                TTFF = np.nan
                TFD = np.nan
                count = np.nan

            df.loc[index, f'iam_standard_{stim}_Saliency_{aoi}_TTFF']=TTFF
            df.loc[index, f'iam_standard_{stim}_Saliency_{aoi}_TFD']=TFD
            df.loc[index, f'iam_standard_{stim}_Saliency_{aoi}_FixationCount']=count


C:\Users\ashra\AppData\Local\Temp\ipykernel_18440\3820113510.py:23: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df.loc[index, f'iam_standard_{stim}_Saliency_{aoi}_FixationCount']=count
C:\Users\ashra\AppData\Local\Temp\ipykernel_18440\3820113510.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df.loc[index, f'iam_standard_{stim}_Saliency_{aoi}_TTFF']=TTFF
C:\Users\ashra\AppData\Local\Temp\ipykernel_18440\3820113510.py:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.inse

In [12]:
# Calculate BP data for Standard

df_bp = df_et.copy().loc[(~df_et.AOI.str.contains('Item'))
                            &(df_et.Stim.str.contains('Stim'))]

# Define top 5 high-gp items

brand_prominence_aois = ['Logo','Perio']

for index, row in df.iterrows():
    for stim in df_bp.Stim.unique():
        try:
            TFD = df_bp.loc[ (df_bp.respondent == row.respondent)
                                &(df_bp.Stim.str.contains(stim))
                                &(df_bp.AOI.str.contains('|'.join(brand_prominence_aois)))]['TFD'].sum()
        except:
            TFD = np.nan

        df.loc[index, f'iam_standard_{stim}_BrandProminence_TFD']=TFD

C:\Users\ashra\AppData\Local\Temp\ipykernel_18440\2447337090.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df.loc[index, f'iam_standard_{stim}_BrandProminence_TFD']=TFD
C:\Users\ashra\AppData\Local\Temp\ipykernel_18440\2447337090.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df.loc[index, f'iam_standard_{stim}_BrandProminence_TFD']=TFD
C:\Users\ashra\AppData\Local\Temp\ipykernel_18440\2447337090.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many tim

In [13]:
# Calculate BP data for Find Anything

df_bp = df_et.copy().loc[(~df_et.AOI.str.contains('Item'))
                            &(df_et.Stim.str.contains('Current | New'))]

# Define top 5 high-gp items

brand_prominence_aois = ['Logo','Perio']
find_anything_stims = {'Slide8': 'FrontPage',
                       'Slide13': 'BackPage'}

for index, row in df.iterrows():
    for stim,label in find_anything_stims.items():
        try:
            TFD = df_bp.loc[ (df_bp.respondent == row.respondent)
                                &(df_bp.Stim.str.contains(stim))
                                &(df_bp.AOI.str.contains('|'.join(brand_prominence_aois)))]['TFD'].sum()
        except:
            TFD = np.nan

        df.loc[index, f'iam_findAnything_{label}_BrandProminence_TFD']=TFD

C:\Users\ashra\AppData\Local\Temp\ipykernel_18440\2492183810.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df.loc[index, f'iam_findAnything_{label}_BrandProminence_TFD']=TFD
C:\Users\ashra\AppData\Local\Temp\ipykernel_18440\2492183810.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df.loc[index, f'iam_findAnything_{label}_BrandProminence_TFD']=TFD


In [14]:
# Calculate find specific heading metrics

df_head = df_et.copy().loc[(df_et.AOI.str.contains('Headin'))
                           &(df_et.Stim.str.contains('Item'))]

for index, row in df.iterrows():
    for stim, df_stim in df_head.groupby('Stim'):
        for aoi in df_stim.AOI.unique():
            try:
                TTFF = df_stim.loc[ (df_stim.respondent == row.respondent)
                                    &(df_stim.AOI == aoi)]['TTFF'].sort_values(ascending=True).values[0]
                TFD = df_stim.loc[ (df_stim.respondent == row.respondent)
                                    &(df_stim.AOI == aoi)]['TFD'].sum()
                count = df_stim.loc[ (df_stim.respondent == row.respondent)
                                    &(df_stim.AOI == aoi)]['TTFF'].count()
            except:
                TTFF = np.nan
                TFD = np.nan
                count = np.nan

            df.loc[index, f'iam_findSpecific_{stim}_Saliency_{aoi}_TTFF']=TTFF
            df.loc[index, f'iam_findSpecific_{stim}_Saliency_{aoi}_TFD']=TFD
            df.loc[index, f'iam_findSpecific_{stim}_Saliency_{aoi}_FixationCount']=count

C:\Users\ashra\AppData\Local\Temp\ipykernel_18440\1415100506.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df.loc[index, f'iam_findSpecific_{stim}_Saliency_{aoi}_TTFF']=TTFF
C:\Users\ashra\AppData\Local\Temp\ipykernel_18440\1415100506.py:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df.loc[index, f'iam_findSpecific_{stim}_Saliency_{aoi}_TFD']=TFD
C:\Users\ashra\AppData\Local\Temp\ipykernel_18440\1415100506.py:23: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` 

In [15]:
# Calculate find anything AOI metrics

df_head = df_et.copy().loc[~(df_et.AOI.str.contains('Item'))]

find_anything_stims = {'Slide8': 'FrontPage',
                       'Slide13': 'BackPage'}

for index, row in df.iterrows():
    for stim,label in find_anything_stims.items():
        for aoi in df_head.loc[df_head.Stim.str.contains(stim)].AOI.unique():
            try:
                TTFF = df_head.loc[ (df_head.respondent == row.respondent)
                                   &(df_head.Stim == stim)
                                    &(df_head.AOI == aoi)]['TTFF'].sort_values(ascending=True).values[0]
                TFD = df_head.loc[ (df_head.respondent == row.respondent)
                                  &(df_head.Stim == stim)
                                    &(df_head.AOI == aoi)]['TFD'].sum()
                count = df_head.loc[ (df_head.respondent == row.respondent)
                                    &(df_head.Stim == stim)
                                    &(df_head.AOI == aoi)]['TTFF'].count()
            except:
                TTFF = np.nan
                TFD = np.nan
                count = np.nan

            df.loc[index, f'iam_findAnything_{label}_Saliency_{aoi}_TTFF']=TTFF
            df.loc[index, f'iam_findAnything_{label}_Saliency_{aoi}_TFD']=TFD
            df.loc[index, f'iam_findAnything_{label}_Saliency_{aoi}_FixationCount']=count

C:\Users\ashra\AppData\Local\Temp\ipykernel_18440\1972683562.py:26: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df.loc[index, f'iam_findAnything_{label}_Saliency_{aoi}_TTFF']=TTFF
C:\Users\ashra\AppData\Local\Temp\ipykernel_18440\1972683562.py:27: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df.loc[index, f'iam_findAnything_{label}_Saliency_{aoi}_TFD']=TFD
C:\Users\ashra\AppData\Local\Temp\ipykernel_18440\1972683562.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert

# GSR

In [16]:
def read_study_txt(path, metadata=None, header_keyword="STUDY", encoding="utf-8"):
    """
    Reads a tab-delimited .txt exported from iMotions (or similar) and
    extracts optional metadata lines that precede the header row.

    Parameters
    ----------
    path : str or Path
        Path to the .txt file.
    metadata : list[str], optional
        Keys to pull out of the metadata section. If empty / None → ignore metadata.
    header_keyword : str, default "STUDY"
        First token in the *header* line. Everything above it is treated as metadata.
    encoding : str, default "utf-8"
        File encoding passed to `open()` and `pd.read_csv()`.

    Returns
    -------
    df : pd.DataFrame
        Data block (tab-delimited, header inferred at runtime).
    meta_dict : dict
        {key: value} pairs for requested metadata fields.
    """
    metadata     = metadata or []      # normalise None → []
    meta_dict    = {}
    meta_lines   = []
    header_index = None                # which line number is the header?
    
    # ---------- 1. Scan once to find the header & stash meta lines ----------
    with open(path, "r", encoding=encoding) as f:
        for i, line in enumerate(f):
            if line.startswith(header_keyword):
                header_index = i
                break
            meta_lines.append(line.rstrip("\n"))
    
    if header_index is None:
        raise ValueError(f"Header row starting with '{header_keyword}' not found in {path}")
    
    # ---------- 2. Parse requested metadata keys ----------
    # Each meta line is assumed "Key<TAB>Value<TAB>Value2..."
    wanted = set(metadata)
    for line in meta_lines:
        parts = line.split("\t")
        if len(parts) > 1:
            key   = parts[0].strip()
            value = "\t".join(parts[1:]).strip()
            if key in wanted:
                meta_dict[key] = value
    
    # ---------- 3. Read the data block with pandas ----------
    df = pd.read_csv(
        path,
        sep="\t",
        header=header_index-3,
        low_memory=False,
        encoding=encoding
    )
    return df, meta_dict

results = []
files = get_files('infiles/IAT/')
for file in files:
    data,_ = read_study_txt(f'infiles/IAT/{file}')
    results.append(data)

df_iat = pd.concat(results)
df_iat.to_csv('results/iat.csv')

In [17]:
# Read in and merge raw ET data

results = []
files = get_files('infiles/GSR/')
for file in files:
    data,_ = read_imotions(f'infiles/GSR/{file}')
    results.append(data)

df_gsr = pd.concat(results)

for res, res_data in df_gsr.groupby('Respondent Name'):
    try:
        df_gsr.loc[df_gsr['Respondent Name']==res, 'resp_id_old'] = int(res.split('_')[1])
    except:
        df_gsr.loc[df_gsr['Respondent Name']==res, 'resp_id_old'] = int(res.split(' ')[1])

df_gsr = df_gsr.rename({'Study Name':'subset_old'},axis = 1)
resp_key = pd.read_csv('infiles/Keys/nandos_resp_id_new.csv')
df_gsr = pd.merge(df_gsr,resp_key, on = ['resp_id_old','subset_old'])

gsr_rename = {
    'Parent Stimulus':'Stim',
    'Peaks Per Minute':'PPM'
}

df_gsr = df_gsr.rename(gsr_rename, axis = 1)




In [18]:
# Get GSR data for find specific and standard exposure tasks
df_gs = df_gsr.copy().loc[(df_gsr.Label.str.contains('Item'))
                          |(df_gsr.Label.str.contains('Stim'))]

slide_map = {
    'Slide8 Current1' : 'findAnything_FrontPage',
    'Slide13 Current2': 'findAnything_BackPage',
    'Slide22 Item1'   : 'findSpecific_Item1',
    'Slide24 Item2'   : 'findSpecific_Item2',
    'Slide26 Item3'   : 'findSpecific_Item3',
    'Slide28 Item4'   : 'findSpecific_Item4',
    'Slide30 Item5'   : 'findSpecific_Item5',
    'Slide63 Stim10'  : 'standard_Stim10',
    'Slide45 Stim4'   : 'standard_Stim4',
    'Slide60 Stim9'   : 'standard_Stim9',
    'Slide42 Stim3'   : 'standard_Stim3',
    'Slide66 Stim11'  : 'standard_Stim11',
    'Slide39 Stim2'   : 'standard_Stim2',
    'Slide57 Stim8'   : 'standard_Stim8',
    'Slide48 Stim5'   : 'standard_Stim5',
    'Slide36 Stim1'   : 'standard_Stim1',
    'Slide54 Stim7'   : 'standard_Stim7',
    'Slide51 Stim6'   : 'standard_Stim6',
    'Slide69 Stim12'  : 'standard_Stim12',
    'Slide8 NewF1'    : 'findAnything_FrontPage',
    'Slide13 NewF2'   : 'findAnything_BackPage',
    'Slide8 NewU1'    : 'findAnything_FrontPage',
    'Slide13 NewU2'   : 'findAnything_BackPage',
}

for index, row in df.iterrows():
    for stim, df_stim in df_gs.groupby('Label'):
            try:
                PPM = df_stim.loc[ (df_stim.respondent == row.respondent)]['PPM'].sort_values(ascending=True).values[0]
            except:
                PPM = np.nan

            label = stim.split(' ')[1]
            df.loc[index, f'gsr_{slide_map[stim]}_PPM']=PPM


C:\Users\ashra\AppData\Local\Temp\ipykernel_18440\3628105912.py:39: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df.loc[index, f'gsr_{slide_map[stim]}_PPM']=PPM
C:\Users\ashra\AppData\Local\Temp\ipykernel_18440\3628105912.py:39: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df.loc[index, f'gsr_{slide_map[stim]}_PPM']=PPM
C:\Users\ashra\AppData\Local\Temp\ipykernel_18440\3628105912.py:39: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performan

# Output final UV

In [19]:
df.to_csv('results/final_uv.csv')

In [10]:
df.to_csv('results/new_uv.csv')